<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/RSI_YUKARI_KES%C4%B0LD%C4%B0M%C4%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Örnek veri seti: 100 günlük kapanış fiyatları
data = {
    "timestamp": pd.date_range(start="2024-01-01", periods=100, freq="D"),
    "LITUSDT": np.linspace(10, 20, 100),   # LIT için örnek artış
    "ADAUSDT": np.linspace(0.5, 2, 100),   # ADA için örnek artış
    "BTC_D": np.linspace(50, 40, 100),     # BTC Dominance düşüş trendi
}

df = pd.DataFrame(data)

# RSI hesaplama fonksiyonu (TA-Lib kullanmadan)
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=period, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=period, min_periods=1).mean()

    rs = avg_gain / (avg_loss + 1e-10)  # Bölme hatalarını önlemek için küçük bir değer eklenir
    rsi = 100 - (100 / (1 + rs))
    return rsi

# RSI hesaplamaları
df["LIT_RSI"] = calculate_rsi(df["LITUSDT"], period=14)
df["ADA_RSI"] = calculate_rsi(df["ADAUSDT"], period=14)
df["BTC_D_RSI"] = calculate_rsi(df["BTC_D"], period=14)

# Kesişimleri belirleme (BTC.D RSI yukarı kesildi mi?)
df["LIT_Cross"] = (df["LIT_RSI"].shift(1) < df["BTC_D_RSI"].shift(1)) & (df["LIT_RSI"] > df["BTC_D_RSI"])
df["ADA_Cross"] = (df["ADA_RSI"].shift(1) < df["BTC_D_RSI"].shift(1)) & (df["ADA_RSI"] > df["BTC_D_RSI"])

# Sonuçları göster
crosses = df[df[["LIT_Cross", "ADA_Cross"]].any(axis=1)][["timestamp", "LIT_Cross", "ADA_Cross"]]
print(crosses)


Empty DataFrame
Columns: [timestamp, LIT_Cross, ADA_Cross]
Index: []
